In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

mofem_um_dir="/home/karol/mofem_install/users_modules"

In [ ]:
# global parameters

max_time=5
ts_MAX_steps=500
enforce_velocity=0
log_file='log'


def run_test(order, penalty, phi, nitsche, mesh="adv3_plast_2D.cub"):
  !make >> /dev/null
  !./ale_plastic_2d \
    -file_name {mesh} \
    -order {order} -penalty {penalty} -phi {phi} -nitsche {nitsche} \
    -base ainsworth -yield_stress 400 \
    -ts_adapt_type basic -ts_dt 0.01 -ts_adapt_dt_max 0.05 -ts_adapt_dt_min 0.0005 -ts_max_time {max_time} \
    -ts_adapt_monitor 0 -ts_max_steps {ts_MAX_steps} -is_ale 1 \
    -snes_type newtonls -snes_linesearch_type bt -snes_linesearch_max_it 3 \
    -ts_type beuler \
    -ts_theta_initial_guess_extrapolate 1 \
    -ts_theta_theta 1 \
    -test_penalty 1 -test_convection 1 -max_load_time 1 -angular_velocity 0.05 -test_H1 0 \
    -snes_atol 1e-6 -snes_rtol 1e-6 -snes_max_it 10 -is_cut_off 0 -snes_fd 0  -enforce_vel_vector {enforce_velocity} 2>&1 | tee {log_file}
    
    # &> overwrites output without showing in the terminal
     
    # process the log file
  !rm -f reaction1  
  !grep "dissipation" log | awk '{print $5 "," $8 "," $10 "," $12}'  &>> reaction1

  return 0

# run_test(2, 1e2, 0, 0)
run_test(2, 1e2, 0, 0)
# run_test(2, 1e2, 0, 0, mesh="adv3_plast_2D_fine.cub")

#   l2_norm =!grep L2 log | tail -n 1 | awk '{print $6}'
#   l2_norm = float(l2_norm[0])
#   energy_norm =!grep Energetic log | awk '{print $6}'
#   energy_norm = float(energy_norm[0])
#   h1_norm =!grep H1 log | awk '{print $6}'
#   h1_norm = float(h1_norm[0])       
#   return [l2_norm, energy_norm, h1_norm]
  
# [l2_norm, energy_norm, h1_norm] = run_test(1, 1e2, -1, 0)
# print('L2 norm %6.4e ' % l2_norm)
# print('Energy norm %6.4e ' % energy_norm)
# print('H1 norm %6.4e ' % h1_norm)

#!{mbconvert} out_result.h5m out_result.vtk

In [ ]:
# playing with 

reaction_data=pd.read_csv('reaction1', header=None)
reaction_data=reaction_data.rename(columns={0: "Time", 1: "energy", 2: "diss", 3: 'eta'})
# print(reaction_data)

# reaction_data=reaction_data.drop([0, 1, 2, 3], axis=1)


# reaction_data=reaction_data[4].str.split(' ', expand=True)
# reaction_data=reaction_data.drop([1, 2, 4, 6], axis=1)
# reaction_data=reaction_data.rename(columns={0: "Time", 3: "X", 5: "Y", 7: 'Z'})

mult_factor=1e4

reaction_data['Time'] = reaction_data['Time'].astype(float)
reaction_data['energy'] = reaction_data['energy'].astype(float)
reaction_data['diss'] = reaction_data['diss'].astype(float)
reaction_data['eta'] = reaction_data['eta'].astype(float)

# reaction_data['D']=reaction_data['Time'].to_numpy()*radius1 # indentation depth
plt.rcParams['figure.figsize'] = [15,5]
fig, ax = plt.subplots(1,3)

# plt.title('Reaction Indenter')
# plt.ylabel('Time')
# plt.xlabel('Indenter depth')
# plt.grid(True)
# plt.legend()

ax[0].plot(reaction_data['Time'].to_numpy(),reaction_data['energy'].to_numpy()*mult_factor,'r^-', label='energy')
ax[1].plot(reaction_data['Time'].to_numpy(),reaction_data['diss'].to_numpy()*mult_factor,'g^-', label='dissipation')
ax[2].plot(reaction_data['Time'].to_numpy(),reaction_data['eta'].to_numpy(),'b^-', label='eta')

for a in range(3):
  ax[a].grid(True)
  ax[a].legend(loc='upper right', shadow=True, fontsize='large')

ax[2].legend(loc='lower right', shadow=True, fontsize='large')

# plt.show()
plt.savefig("test_with_enforce_vel_{}.jpg".format(enforce_velocity), dpi=150)

In [ ]:
!{mofem_um_dir}/tools/convert.py -np 12 out_*h5m